# Tesseract

In [153]:
path = 'new.png'

In [89]:
from whitening import whiten
from PIL import Image
from PIL import ImageChops

In [90]:
image = Image.open(path)
image, background = whiten(image, kernel_size=20, downsample=5)
image = ImageChops.subtract(image,background)
mask1 = Image.eval(image, lambda a: 0 if a <= 24 else 255)
mask2 = mask1.convert('1')

blank = Image.eval(image, lambda a: 0)

new = Image.composite(background, blank, mask2) 
# new.show()

In [91]:
new.save('test.png','PNG')

# tesseract without any segmentation 

In [116]:
from tesserocr import PyTessBaseAPI , RIL
import tesserocr
img=Image.open(path)
img=img.resize((200,200))
api = PyTessBaseAPI(lang='eng')
api.SetImage(img)
print ( "OCR Process begin .....")
# start_time = time.time()
api.SetVariable("tessedit_char_whitelist", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
b = api.GetUTF8Text()

OCR Process begin .....


# Tesseract with PSM 


In [142]:
img=Image.open(path)
# img=img.resize((400,200))
api = PyTessBaseAPI(lang='eng', psm=5)
api.SetImage(img)
print ( "OCR Process begin .....")
# start_time = time.time()
api.SetVariable("tessedit_char_whitelist", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
b = str(api.GetUTF8Text()).splitlines()
b

OCR Process begin .....


['=']

# Tesseract with image RIL 

In [156]:
from PIL import Image
from tesserocr import PyTessBaseAPI, RIL

image = Image.open(path)
with PyTessBaseAPI() as api:
    api.SetImage(image)
    boxes = api.GetComponentImages(RIL.PARA, True)
    print('Found {} textline image components.'.format(len(boxes)))
    for i, (im, box, _, _) in enumerate(boxes):
        # im is a PIL image object
        # box is a dict with x, y, w and h keys
        api.SetRectangle(box['x'], box['y'], box['w'], box['h'])
        ocrResult = api.GetUTF8Text()
        conf = api.MeanTextConf()
        print(u"Box[{0}]: x={x}, y={y}, w={w}, h={h}, "
              "confidence: {1}, text: {2}".format(i, conf, ocrResult, **box))

Found 0 textline image components.


In [160]:
from __future__ import print_function

from tesserocr import PyTessBaseAPI, RIL, iterate_level

with PyTessBaseAPI() as api:
    api.SetImageFile(path)
    api.SetVariable("save_blob_choices", "T")
    api.SetRectangle(50, 50 , 121, 186)
    api.Recognize()

    ri = api.GetIterator()
    level = RIL.SYMBOL
    for r in iterate_level(ri, level):
        symbol = r.GetUTF8Text(level)  # r == ri
        conf = r.Confidence(level)
        if symbol:
            print(u'symbol {}, conf: {}'.format(symbol, conf), end='')
        indent = False
        ci = r.GetChoiceIterator()
        for c in ci:
            if indent:
                print('\t\t ', end='')
            print('\t- ', end='')
            choice = c.GetUTF8Text()  # c == ci
            print(u'{} conf: {}'.format(choice, c.Confidence()))
            indent = True
        print('---------------------------------------------')

RuntimeError: No text returned

In [4]:
#read image
import cv2
img = cv2.imread(r"D:\tapway\Plate_detect_and_recognize\testcase\cropped-lp1-20200715-104932371Z.jpg")

#grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.waitKey(0)

#binarize 
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
cv2.waitKey(0)

#find contours
ctrs, hier = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, 
cv2.CHAIN_APPROX_SIMPLE)

#sort contours
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

for i, ctr in enumerate(sorted_ctrs):
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)

    # Getting ROI
    roi = img[y:y+h, x:x+w]

    # show ROI
    #cv2.imwrite('roi_imgs.png', roi)
    cv2.imshow('charachter'+str(i), roi)
    cv2.rectangle(img,(x,y),( x + w, y + h ),(90,0,255),2)
    cv2.waitKey(0)

cv2.imshow('marked areas',img)
cv2.waitKey(0)

-1

In [1]:
import cv2
img = cv2.imread(r"D:\tapway\testcase\cropped-lp1-20200715-110252772Z.jpg", 0)
cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU,img)
contours, hier = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
cv2.imshow("contours", img)
cv2.waitKey(0)
d=0
for ctr in contours:
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)
    # Getting ROI
    roi = img[y:y+h, x:x+w]

    cv2.imshow('character: %d'%d,roi)
    cv2.imwrite('character_%d.png'%d, roi)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    d+=1